# Setup Omero

This notebook goes through checking if Omero can connect to this project through Python. You will need the hostname, your username and password to access it.

In [18]:
# Function to connect to the Omero server
# Might clean this up into a separate py file if more is added to this notebook

from omero.gateway import BlitzGateway

# Function that connects the user to the omero server
def connect(hostname, username, password):
    """
    Connect to an OMERO server
    :param hostname: Host name
    :param username: User
    :param password: Password
    :return: Connected BlitzGateway
    """
    conn = BlitzGateway(username, password,
                        host=hostname, secure=True)
    conn.connect()
    conn.c.enableKeepAlive(60)
    return conn

# Using the function from above
HOST = 'ome2.hpc.sussex.ac.uk' #change if different
conn = connect(HOST, input("Username: "), getpass("OMERO Password: "))
try:
    print('Connected as', conn.getUser().getName())
except AttributeError:
    print('You inputted a wrong username and/or password.')

You inputted a wrong username and/or password.
